# RESUMEN PANDAS

In [2]:
import pandas as pd

### Creacion de un dataframe

In [3]:
dfo = pd.DataFrame([1.5, 1.6, 1.75, 1.80], index=['Jane', 'Joe', 'Susan', 'Mike'])

population_dict = {'California': 38332521,
                   'Texas': 26448193,
                   'New York': 19651127,
                   'Florida': 19552860,
                   'Illinois': 12882135}
population = pd.Series(population_dict) 

lista = ['Pepe', 'Pedro', 'Jose']
df_lista = pd.DataFrame(lista)

data = {
    'Ciudad': ['Madrid', 'Barcelona', 'Valencia', 'Sevilla', 'Zaragoza'],
    'Población': [3223334, 1620343, 791413, 688711, 666880],
    'Año de Fundación': [860, -15, 138, 712, -14],
}
df = pd.DataFrame(data)

### Propiedades

In [4]:
df.columns # columnas
df.index # indices
df.values # Devuelve valores
df.keys()
df.T # traspuesta
df.shape

(5, 3)

### Merge

![imagen](./01-Teoría/img/merges.png)

In [5]:
# Creamos unos dataframes de ejemplo:
data_peliculas = {
    'Titulo': ['Inception', 'The Dark Knight', 'Interstellar', 'Pulp Fiction', 'Kill Bill: Vol. 1'],
    'Director': ['Christopher Nolan', 'Christopher Nolan', 'Christopher Nolan', 'Quentin Tarantino', 'Quentin Tarantino'],
    'Año': [2010, 2008, 2014, 1994, 2003]
}
peliculas_df = pd.DataFrame(data_peliculas)

data_directores = {
    'Director': ['Christopher Nolan', 'Quentin Tarantino', 'Martin Scorsese'],
    'Nacionalidad': ['Reino Unido', 'Estados Unidos', 'Estados Unidos'],
    'Nacimiento': [1970, 1963, 1942]
}
directores_df = pd.DataFrame(data_directores)

# En terminos graficos el orden de como pongamos los dataframes sera el orden derecha e izquierda, respectivamente, del merge
merged_df = pd.merge(peliculas_df, directores_df,  # Aqui los dataframes que queremos combinar
    on=['Director'], # Esto puede ser una lista para poder combinar varias columnas
    how='outer', # Indica la forma del merge, en la imagen se ven las diferencias, es outer de forma default
    indicator=True) # Creara una columna _merge que indica si 
merged_df

# Imaginemos que director no se llama director en alguna de las dos:
peliculas_df.columns = ['Titulo', 'Direct', 'Año']
merged_df2 = pd.merge(peliculas_df, directores_df,  # Aqui los dataframes que queremos combinar
    left_on=['Direct'], right_on=['Director'],
    how='inner',
    indicator=True) # El problema es que las columnas se podrian llegar a duplicar
merged_df2 # Preguntad Jonatan porque no se unen Direct y Director

# Recordamos que podemos hacer merge de multiples columnas, con una lista

,Titulo,Direct,Año,Director,Nacionalidad,Nacimiento,_merge
0,Inception,Christopher Nolan,2010,Christopher Nolan,Reino Unido,1970,both
1,The Dark Knight,Christopher Nolan,2008,Christopher Nolan,Reino Unido,1970,both
2,Interstellar,Christopher Nolan,2014,Christopher Nolan,Reino Unido,1970,both
3,Pulp Fiction,Quentin Tarantino,1994,Quentin Tarantino,Estados Unidos,1963,both
4,Kill Bill: Vol. 1,Quentin Tarantino,2003,Quentin Tarantino,Estados Unidos,1963,both


## Analisis de datos

In [6]:
terremotos = pd.read_csv('./01-Teoría/data/earthquakes.csv', 
    sep=',', # De forma default sera la coma
    encoding="utf-8" # utf-8 es el default, "latin-1" es el castellano
)

terremotos.info() # detalles de tipos y memoria
terremotos.describe() # detalles estadisticos
terremotos.tail() # Mostrar n ultimas filas
terremotos.head(2) # Mostar n primeras filas
terremotos.sample(2) # Muestra aleatoria

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9332 entries, 0 to 9331
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mag           9331 non-null   float64
 1   magType       9331 non-null   object 
 2   time          9332 non-null   int64  
 3   place         9332 non-null   object 
 4   tsunami       9332 non-null   int64  
 5   parsed_place  9332 non-null   object 
dtypes: float64(1), int64(2), object(3)
memory usage: 437.6+ KB


,mag,magType,time,place,tsunami,parsed_place
1913,4.9,mb,1538872003570,"67km SSW of Intipuca, El Salvador",0,El Salvador
5896,1.6,ml,1538009428474,"123km NNW of Arctic Village, Alaska",0,Alaska


### Operaciones basicas

In [7]:
terremotos['mag'].sum() # Suma todos los valores
terremotos['mag'].count() # Cuenta cuantos hay, interesante en booleanos
len(terremotos['mag']) # Esto no es lo mismo porque el count no cuenta Nans
terremotos['mag'].max()
terremotos['mag'].min()
terremotos['mag'].median() # mediana
terremotos['mag'].mean() # media
terremotos['mag'].mode() # moda
terremotos['mag'].std() # desviacion tipica
terremotos['mag'].unique() # Muestra los valores unicos
terremotos['mag'].duplicated() # Valores duplicados
terremotos['mag'].quantile(0.95) # percentiles
terremotos['mag']*1 # multriplica todos, cualquier operacion es valida

0       1.35
1       1.29
2       3.42
3       0.44
4       2.16
        ... 
9327    0.62
9328    1.00
9329    2.40
9330    1.10
9331    0.66
Name: mag, Length: 9332, dtype: float64

### Mascaras

In [8]:
# Podemos poner tantas condiciones como queramos 
resultado = terremotos[(terremotos['parsed_place'] == 'Japan') & (terremotos['mag'] >= 4.9)]
# Basicamente buscamos dentro del dataframe terremotos todos los terremotos de japon con una magnitud igual o mayor a 4.9

resultado1 = terremotos[terremotos['mag'] == terremotos['mag'].max()]
# Buscamos todos los terremotos con una escala maxima

# Importante: los operadores de las mascaras siempre son |, &, == y nunca cosas como and o or

### Funciones importantes

In [9]:
faang = pd.read_csv('./01-Teoría/data/faang.csv')

# groupby: elemento que crea agrupaciones para hacer operaciones
faang.groupby(['ticker'])['volume'].sum()
# lo que hace groupby es basicamente es agrupar por elementos unicos, en este caso estamos agrupando por ticker,
# que tiene las cateogiras aapl, amzn, fb, goog, nflx y separa cada uno y coge el volumen de cada categoria, y luego
# hemos hecho un sum, devolviendonos la suma de todos los volumenes de cada categoría.

# apply: operaciones linea por linea
faang['apertura top'] = faang['open'].apply(lambda x: x > 1400)
# Imaginemos como la x a cada row de open. Hemos creado una columna donde por cada fila de open 
# Comprobamos si el elemento es mayor a 1400 y devuelve un booleano y lo añade a esa misma fila pero 
# con la nueva columna

# transform: parecido al apply pero podemos hacer operaciones a grupos
faang['dist_media'] = faang.groupby(['ticker'])['open'].transform(lambda x: x - x.mean())
# lo que hacemos y por lo que se diferencia del apply, es que estamos creando una columna, operando linea por linea
# pero en este caso la x es cada resultado de open separado por categoria, de esta forma podemos operar individualmente
# la x y acceder a x.mean() que en este caso es la media pero de todo open de cada categoria, al devolver el resultado
# lo coloca en la fila donde ha cogido la informacion, es decir cuando se accede al dataframe sin el groupby, se coloca
# donde corresponde.

# falta aggregate

### Loc e Iloc

In [10]:
# se utiliza el método .loc[] para filtrar las filas y seleccionar las columnas al mismo tiempo
todo_en_uno = terremotos[terremotos['parsed_place'] == 'Japan'][['parsed_place', 'mag', 'magType']]
otra = terremotos.loc[terremotos['parsed_place'] == 'Japan', ['parsed_place', 'mag', 'magType']]
# En este codigo ambos hacen lo mismo, seleccionan todas las filas del dataframe terremoto donde el lugar sea japon
# y luego seleccionan las columnas lugar, magnitud y tipo de magnitud y crea un dataframe de esto,
# pero .loc[] es útil cuando necesitas seleccionar datos de manera más precisa, cuando las condiciones de selección 
# son más complejas o cuando deseas realizar operaciones de asignación de manera segura.

# El método .iloc[] se utiliza para seleccionar datos en función de sus posiciones numéricas en el DataFrame.
subset = terremotos.iloc[:5, :3]
# :5 indica que queremos seleccionar desde la primera fila hasta la quinta fila (exclusiva).
# :3 indica que queremos seleccionar desde la primera columna hasta la tercera columna

# es útil cuando necesitas seleccionar datos en función de su posición numérica en lugar de sus etiquetas.
# Esto es especialmente útil cuando trabajas con grandes conjuntos de datos y no necesariamente conoces las etiquetas 
# de las filas o columnas, pero sabes su posición relativa dentro del DataFrame.



### NaN

In [11]:
# ¿hay algun nan en el dataframe?
terremotos.isna().any() # podemos usar isna() o isnull()

# magType y mag son las unicas columnas con Nan
# Eliminamar el Nan
terremotos.dropna(subset=['mag'], inplace=True) # El inplace true para que modifique el dataframe directamente

# Rellenar los Nans
terremotos['magType'] = terremotos['magType'].fillna("")
terremotos

,mag,magType,time,place,tsunami,parsed_place
0,1.35,ml,1539475168010,"9km NE of Aguanga, CA",0,California
1,1.29,ml,1539475129610,"9km NE of Aguanga, CA",0,California
2,3.42,ml,1539475062610,"8km NE of Aguanga, CA",0,California
3,0.44,ml,1539474978070,"9km NE of Aguanga, CA",0,California
4,2.16,md,1539474716050,"10km NW of Avenal, CA",0,California
...,...,...,...,...,...,...
9327,0.62,md,1537230228060,"9km ENE of Mammoth Lakes, CA",0,California
9328,1.00,ml,1537230135130,"3km W of Julian, CA",0,California
9329,2.40,md,1537229908180,"35km NNE of Hatillo, Puerto Rico",0,Puerto Rico
9330,1.10,ml,1537229545350,"9km NE of Aguanga, CA",0,California


### Index

In [12]:
faang = pd.read_csv('./01-Teoría/data/faang.csv')

# Creamos un indice
faang.set_index('date', inplace=True) # Es una buena practica establecer la fecha como indice, comprobando antes que no hay cosas como duplicados

# Si queremos quitar el indice
faang.reset_index(inplace=True)
faang.head(2)

,date,Unnamed: 0,high,low,open,close,volume,ticker
0,2018-01-02,0,43.075001,42.314999,42.540001,43.064999,102223600.0,aapl
1,2018-01-03,1,43.637501,42.990002,43.132500,43.057499,118071600.0,aapl


## Procesos tipicos

In [15]:
# columnas
faang.columns
faang.columns = [x.lower() for x in faang.columns] # cambiamos las columnas
faang.rename(columns={'open': 'Open'}, inplace=True) # Otra forma

faang.drop(columns=["unnamed: 0"], inplace=True, errors='ignore') # Eliminar columnas ignorando errores

# Averiguamos el tipo de dato de la columna date
faang['date'].dtypes
type(faang['date'][0])
faang['date'] = pd.to_datetime(faang['date']) # pasamos a formato fecha

# Para cambiar el tipo de dato de una columna (en este caso lo pasamos a int y luego lo volvemos a dejar en float)
terremotos['mag'] = terremotos['mag'].astype(str).astype(float)

# Comprobamos si existen duplicados: dos formas, importante hacerlo en columnas donde el valor debe ser unico (no es el caso del codigo de abajo)
faang['date'].value_counts() # En forma de dataserie, esto es interesante tambien si solo hay dos valores y queremos ver cuantos son true y cuantos falses (por ejemplo)
faang[faang["date"].duplicated()] # En forma de dataframe

# Ordenar
faang_ordenado = faang.sort_index() # Ordena por indice de forma ascendente por defecto
faang_ordenado = faang.sort_index(ascending=False) # esto es de mayor a menor
faang_ordenado = faang.sort_values(by='high' ,ascending=True) # ascending es true de forma default
